In [18]:
import pandas as pd
import numpy as np
suffix=f'先进制造未上市'
doc_lda_w2v_0=pd.read_parquet(f'data\\documents_vec_{suffix}.parquet')  

suffix=f'先进制造上市'#,'制造业上市'
doc_lda_w2v_1=pd.read_parquet(f'data\\documents_vec_{suffix}.parquet')

doc_lda_w2v=pd.concat([doc_lda_w2v_0,doc_lda_w2v_1])

patent_co=pd.read_csv('data\patent_co_1953_16k_202407.csv',index_col=0)
patent_co['申请日']=pd.to_datetime(patent_co['申请日'])
patent_co_1=patent_co[(patent_co.行业=='先进制造')]

df_patent_invest=pd.read_csv('data\\patent_invest_202407_发明授权_发明申请.csv',index_col=0)#patent_invest_202408_先进制造
df_patent_invest['融资时间']=pd.to_datetime(df_patent_invest['融资时间'])
final_invest_data_tech_1=df_patent_invest[(df_patent_invest.行业=='先进制造')].sort_values(by=['公司序号','融资时间'])#&(final_invest_data_tech.上市Days.isna())
final_invest_data_tech_1.shape

In [4]:
doc_lda_w2v.shape,patent_co_1.shape

((1348153, 250), (1203317, 39))

In [18]:
doc_lda_w2v.index.value_counts()

序号
CN106370313B    4
CN104183435B    4
CN204067280U    4
CN204067282U    4
CN204067283U    4
               ..
CN105244535B    1
CN105239082B    1
CN105226318B    1
CN105244543B    1
CN3044990D      1
Name: count, Length: 1171534, dtype: int64

In [5]:
doc_lda_w2v=doc_lda_w2v.groupby(doc_lda_w2v.index).mean()
doc_lda_w2v.shape

(1171534, 250)

In [37]:
set(doc_lda_w2v.index)-set(patent_co_1['公开（公告）号'])

set()

In [7]:
final_invest_data_tech_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107145 entries, 20629 to 127770
Data columns (total 62 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   公司序号             107145 non-null  int64         
 1   公司简称             107145 non-null  object        
 2   工商全称             106926 non-null  object        
 3   成立时间             107145 non-null  object        
 4   公司简介             107145 non-null  object        
 5   省                107145 non-null  object        
 6   市                107145 non-null  object        
 7   行业               107145 non-null  object        
 8   子行业              107145 non-null  object        
 9   估算估值             99741 non-null   float64       
 10  北上广深             107145 non-null  int64         
 11  IPO上市日期          26292 non-null   object        
 12  首次上市日期           24372 non-null   object        
 13  目前已上市            107145 non-null  int64         
 14  高科技企业            1071

In [17]:
final_invest_data_tech_1['观察日']=np.where(final_invest_data_tech_1.融资时间.notna(), final_invest_data_tech_1.融资时间,
                                         pd.to_datetime(final_invest_data_tech_1['观察年'], format='%Y')+ pd.offsets.YearEnd())
final_invest_data_tech_1['观察日'].isna().sum()

0

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
tech_category

1721380    F21
1739616    F21
1739617    H05
Name: IPC主分类, dtype: object

In [65]:
 np.empty(len(base_category))

array([0., 0., 0., 0.])

In [67]:
patent_co_1['技术大类']=patent_co_1.IPC主分类.str.strip().str[:3]

C:\Users\Prime\AppData\Local\Temp\ipykernel_24800\1958649803.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patent_co_1['技术大类']=patent_co_1.IPC主分类.str.strip().str[:3]


In [ ]:
%%time
tuple_list = [(1, 4), (2, 4), (1, 5), (2, 5), (2, 6), (3, 4), (3, 5)]
Threshold=0.8
Year=None
for date in sorted(final_invest_data_tech_1['观察日'].unique()):
    if date.year!=Year:
        print(date.year)
        Year=date.year
        
    fiter_invest=final_invest_data_tech_1[final_invest_data_tech_1['观察日']==date]
    filter_value=patent_co_1[patent_co_1.申请日<date]
    if filter_value.empty:
        continue
    
    for index,row in fiter_invest.iterrows():
        filter_value_0=filter_value[filter_value.工商全称==row.工商全称]
        if filter_value_0.empty:
            continue
            
        metrics = {}
        for m,n in tuple_list:
            suffix=f'{m,n}'
            M=m*365
            N=n*365
            filter_value_1=filter_value_0[filter_value_0.申请日>=date-pd.Timedelta(days=N)]
            if filter_value_1.empty:
                continue
                
            mask=filter_value_1.申请日 <  date-pd.Timedelta(days=(N - M))
            tech_category_value = filter_value_1[mask]
            base_category_value = filter_value_1[~mask]
            
            tech_category= doc_lda_w2v.loc[doc_lda_w2v.index.isin(tech_category_value['公开（公告）号'])]
            base_category= doc_lda_w2v.loc[doc_lda_w2v.index.isin(base_category_value['公开（公告）号'])]
            if not tech_category.empty and not base_category.empty:
                cosine_sim = cosine_similarity(tech_category.values,base_category.values)#shape:len(tech_category) x len(base_category)
                tech_base_i = cosine_sim.mean(axis=0) #len(base_category)
                innovation_patent_num = (tech_base_i < Threshold).sum() #统计base_category中小于threshold的专利个数
                
                metrics[f'innovation_patent_num_{suffix}']=innovation_patent_num
                metrics[f'tech_closeness_innovation_{suffix}']=innovation_patent_num / len(filter_value_1)
                metrics[f'tech_base_i_mean_{suffix}']= tech_base_i.mean()      
    
        #--------
            diff_ipcs=set(tech_category_value.技术大类)^set(base_category_value.技术大类)
            innovation_patent_num=len(diff_ipcs)
            
            metrics[f'innovation_patent_num_ipc_{suffix}']=innovation_patent_num
            metrics[f'ipc_innovationc_{suffix}']=innovation_patent_num / len(filter_value_1)
           
            metrics[f'overlap_patent_num_{suffix}'] = len(set(base_category.index) & set(filter_value_1.loc[filter_value_1.技术大类.isin(diff_ipcs),'公开（公告）号']))# 计算 tech_base_i 和 IPC 分类差集的重合部分
    
            
        final_invest_data_tech_1.loc[index, metrics.keys()] = pd.Series(metrics) 


    
final_invest_data_tech_1

1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017


In [72]:
pd.Series(metrics) 

innovation_patent_num_(1, 4)        35589.000000
tech_closeness_innovation_(1, 4)        0.690003
tech_base_i_mean_(1, 4)                 0.731518
innovation_patent_num_ipc_(1, 4)       25.000000
ipc_innovationc_(1, 4)                  0.000485
overlap_patent_num_(1, 4)             258.000000
innovation_patent_num_(2, 4)        19597.000000
tech_closeness_innovation_(2, 4)        0.379949
tech_base_i_mean_(2, 4)                 0.734651
innovation_patent_num_ipc_(2, 4)       18.000000
ipc_innovationc_(2, 4)                  0.000349
overlap_patent_num_(2, 4)              64.000000
innovation_patent_num_(1, 5)        49244.000000
tech_closeness_innovation_(1, 5)        0.760220
tech_base_i_mean_(1, 5)                 0.735229
innovation_patent_num_ipc_(1, 5)       20.000000
ipc_innovationc_(1, 5)                  0.000309
overlap_patent_num_(1, 5)             167.000000
innovation_patent_num_(2, 5)        35281.000000
tech_closeness_innovation_(2, 5)        0.544662
tech_base_i_mean_(2,

In [77]:
final_invest_data_tech_1['成立时间']=pd.to_datetime(final_invest_data_tech_1['成立时间'])
final_invest_data_tech_1['成立到首次融资']=(final_invest_data_tech_1.groupby('公司序号')['融资时间'].transform('min')-final_invest_data_tech_1['成立时间']).dt.days

In [78]:
final_invest_data_tech_1.to_excel('data/patent_innovation_202407_先进制造.xlsx')

In [9]:
final_invest_data_tech_1.columns.to_list()

['公司序号',
 '公司简称',
 '工商全称',
 '成立时间',
 '公司简介',
 '省',
 '市',
 '行业',
 '子行业',
 '估算估值',
 '北上广深',
 'IPO上市日期',
 '首次上市日期',
 '目前已上市',
 '高科技企业',
 '经度',
 '纬度',
 '融资时间',
 '轮次',
 '金额',
 '币种',
 '等值人民币（万）',
 '投资机构',
 'VC_Amount',
 'Cum_invset',
 'Age',
 'VC-num',
 '融资阶段_初创/商业化',
 '有披露',
 'vc_first',
 'vc_distance',
 'VC行业经验',
 'vc_mktexp',
 'vc_rep',
 '成立年份',
 '融资年份',
 '上市Days',
 '观察年',
 '是否融资',
 '投资时间_Shift',
 'Application_num',
 'IPC主分类_去重',
 'IPC主分类_平均',
 'IPC_个数_去重',
 'IPC_个数_平均',
 '知识宽度',
 '被引证次数',
 '引证数量',
 '引证数量_去重',
 '引证数量_平均',
 '简单同族个数',
 '简单同族_去重',
 '简单同族美日欧_比例',
 '发明人数量',
 '发明人数量_去重',
 '发明人数量_平均',
 '权利要求数量',
 '引证科技文献数量',
 '有效数量',
 '有效_比例',
 '发明授权',
 '发明申请',
 '观察日',
 'innovation_patent_num_ipc_(1, 4)',
 'ipc_innovationc_(1, 4)',
 'innovation_patent_num_ipc_(2, 4)',
 'ipc_innovationc_(2, 4)',
 'innovation_patent_num_ipc_(1, 5)',
 'ipc_innovationc_(1, 5)',
 'innovation_patent_num_ipc_(2, 5)',
 'ipc_innovationc_(2, 5)',
 'innovation_patent_num_ipc_(2, 6)',
 'ipc_innovationc_(2, 6)',
 'innovation

In [10]:
fields = [
    # (1, 4)
    [
        'innovation_patent_num_ipc_(1, 4)',
        'ipc_innovationc_(1, 4)',
        'innovation_patent_num_(1, 4)',
        'tech_closeness_innovation_(1, 4)',
        'tech_base_i_mean_(1, 4)',
        'overlap_patent_num_(1, 4)'
    ],
    # (2, 4)
    [
        'innovation_patent_num_ipc_(2, 4)',
        'ipc_innovationc_(2, 4)',
        'innovation_patent_num_(2, 4)',
        'tech_closeness_innovation_(2, 4)',
        'tech_base_i_mean_(2, 4)',
        'overlap_patent_num_(2, 4)'
    ],
    # (3, 4)
    [
        'innovation_patent_num_ipc_(3, 4)',
        'ipc_innovationc_(3, 4)',
        'innovation_patent_num_(3, 4)',
        'tech_closeness_innovation_(3, 4)',
        'tech_base_i_mean_(3, 4)',
        'overlap_patent_num_(3, 4)'
    ],
    # (1, 5)
    [
        'innovation_patent_num_ipc_(1, 5)',
        'ipc_innovationc_(1, 5)',
        'innovation_patent_num_(1, 5)',
        'tech_closeness_innovation_(1, 5)',
        'tech_base_i_mean_(1, 5)',
        'overlap_patent_num_(1, 5)'
    ],
    # (2, 5)
    [
        'innovation_patent_num_ipc_(2, 5)',
        'ipc_innovationc_(2, 5)',
        'innovation_patent_num_(2, 5)',
        'tech_closeness_innovation_(2, 5)',
        'tech_base_i_mean_(2, 5)',
        'overlap_patent_num_(2, 5)'
    ],
    # (3, 5)
    [
        'innovation_patent_num_ipc_(3, 5)',
        'ipc_innovationc_(3, 5)',
        'innovation_patent_num_(3, 5)',
        'tech_closeness_innovation_(3, 5)',
        'tech_base_i_mean_(3, 5)',
        'overlap_patent_num_(3, 5)'
    ],
    # (2, 6)
    [
        'innovation_patent_num_ipc_(2, 6)',
        'ipc_innovationc_(2, 6)',
        'innovation_patent_num_(2, 6)',
        'tech_closeness_innovation_(2, 6)',
        'tech_base_i_mean_(2, 6)',
        'overlap_patent_num_(2, 6)'
    ]
]

In [14]:
final_invest_data_tech_1['是否首轮融资']=(final_invest_data_tech_1['融资时间'] == final_invest_data_tech_1.groupby('公司序号')['融资时间'].transform('min')).astype(int)
final_invest_data_tech_1['是否首轮融资'].sum()

10018

In [16]:
final_invest_data_tech_1.filter(like='融资')

,融资时间,融资阶段_初创/商业化,融资年份,是否融资,成立到首次融资,是否首轮融资
20629,2013-03-01,初创融资阶段,2013.0,1,731,1
20630,2014-09-01,初创融资阶段,2014.0,1,731,0
20627,NaT,NaN,NaN,0,731,0
20628,NaT,NaN,NaN,0,731,0
20632,2012-04-04,初创融资阶段,2012.0,1,247,1
...,...,...,...,...,...,...
127766,NaT,NaN,NaN,0,3561,0
127767,NaT,NaN,NaN,0,3561,0
127768,NaT,NaN,NaN,0,3561,0
127769,NaT,NaN,NaN,0,3561,0
